In [28]:
import pandas as pd

# 전처리 완료한 데이터
df = pd.read_csv('preprocessing_data.csv')

X = df.drop(columns=['100ml당 가격'])
y = df['100ml당 가격']

print(f"30ml 기준 평균가격: {round(df['100ml당 가격'].mean() / 100 * 30)}원")

30ml 기준 평균가격: 54305원


# 머신러닝

In [29]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [30]:
# 예측
y_pred = model.predict(X_test)

# 평가
print("MSE:", mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))

print(f"30ml 기준 평균오차: {round(mean_squared_error(y_test, y_pred) ** 0.5 / 100 * 30)}원")

MSE: 19194601247.542618
R²: 0.32810551848108394
30ml 기준 평균오차: 41563원


In [ ]:
# 모델 저장
joblib.dump(model, "first_machine.pkl")

['first_machine.pkl']

In [23]:
# 모델 불러오기
model = joblib.load('first_machine.pkl')
y_pred = model.predict(X_test)

In [48]:
# 가격예측1
# 개수 1개, 평점 4.5, 리뷰수 1000개, 향/냄새 만족도 80%, 지속력 만족도 80%
# 총 용량 30ml, 공용향수
input_data = pd.DataFrame([[1, 4.5, 1000, 80, 80, 30, 1.0, 0.0]],
                          columns=['개수', '평점', '리뷰수', '향/냄새 만족도(%)', '지속력 만족도(%)', '총 용량', '타겟_공용', '타겟_남성'])
print(f"{int(model.predict(input_data)[0] / 100 * 30)}원")

# 가격예측2
# 개수 1개, 평점 4.0, 리뷰수 200개, 향/냄새 만족도 70%, 지속력 만족도 80%
# 총 용량 60ml, 여성향수
input_data = pd.DataFrame([[1, 4, 200, 70, 80, 60, 0.0, 0.0]],
                          columns=['개수', '평점', '리뷰수', '향/냄새 만족도(%)', '지속력 만족도(%)', '총 용량', '타겟_공용', '타겟_남성'])
print(f"{int(model.predict(input_data)[0] / 100 * 30)}원")

# 가격예측3
# 개수 3개, 평점 4.7, 리뷰수 2500개, 향/냄새 만족도 86%, 지속력 만족도 90%
# 총 용량 30ml, 남성향수
input_data = pd.DataFrame([[3, 4.7, 2500, 86, 90, 30, 0.0, 1.0]],
                          columns=['개수', '평점', '리뷰수', '향/냄새 만족도(%)', '지속력 만족도(%)', '총 용량', '타겟_공용', '타겟_남성'])
print(f"{int(model.predict(input_data)[0] / 100 * 30)}원")

88819원
31407원
90354원


# 딥러닝

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 구성
model = Sequential([
    Dense(32, activation='relu', input_shape=(X.shape[1],)),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1)  # 회귀는 출력 뉴런 1개, 활성화 없음
])

# 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 학습 중단지점 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

# 데이터 분할

# 최적화 모델을 업데이트하고 저장
checkpointer = ModelCheckpoint(filepath="first_deep.keras", monitor='val_loss', verbose=1, save_best_only=True)

# 학습
history = model.fit(X_train, y_train, epochs=3000, batch_size=500, validation_split=0.2, callbacks=[early_stopping_callback,checkpointer])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Streaming output truncated to the last 5000 lines.
Epoch 1351/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 31371462656.0000 - mae: 111505.4766
Epoch 1351: val_loss improved from 47889866752.00000 to 47866568704.00000, saving model to first_deep.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - loss: 31371462656.0000 - mae: 111505.4766 - val_loss: 47866568704.0000 - val_mae: 148781.1562
Epoch 1352/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step - loss: 31354845184.0000 - mae: 111470.6953
Epoch 1352: val_loss improved from 47866568704.00000 to 47843577856.00000, saving model to first_deep.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 723ms/step - loss: 31354845184.0000 - mae: 111470.6953 - val_loss: 47843577856.0000 - val_mae: 148721.1250
Epoch 1353/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step - loss: 31338014720.0000 - mae: 111417.6016
Epoch 1353: val_loss improved from 47843577856.00000 to 47818133504.00000, saving model to first_deep.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 527ms/step - loss: 31338014720.

In [ ]:
# 평가
loss, mae = model.evaluate(X_test, y_test)
print(f"테스트 MSE: {loss}, MAE: {mae}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 23234052096.0000 - mae: 91266.5781 
테스트 MSE: 20820006912.0, MAE: 89983.71875


In [ ]:
print(f"30ml 기준 평균오차: {round(loss ** 0.5 / 100 * 30)}원")

30ml 기준 평균오차: 43287원


In [ ]:
# 모델 저장
model.save("first_deep.keras")

In [49]:
# 모델 불러오기
from tensorflow.keras.models import load_model
model = load_model("first_deep.keras")

In [51]:
# 가격예측1
# 개수 1개, 평점 4.5, 리뷰수 1000개, 향/냄새 만족도 80%, 지속력 만족도 80%
# 총 용량 30ml, 공용향수
input_data = pd.DataFrame([[1, 4.5, 1000, 80, 80, 30, 1.0, 0.0]],
                          columns=['개수', '평점', '리뷰수', '향/냄새 만족도(%)', '지속력 만족도(%)', '총 용량', '타겟_공용', '타겟_남성'])
price1 = model.predict(input_data).item()
print(f"{int(price1 / 100 * 30)}원")

# 가격예측2
# 개수 1개, 평점 4.0, 리뷰수 200개, 향/냄새 만족도 70%, 지속력 만족도 80%
# 총 용량 60ml, 여성향수
input_data = pd.DataFrame([[1, 4.0, 200, 70, 80, 60, 0.0, 0.0]],
                          columns=['개수', '평점', '리뷰수', '향/냄새 만족도(%)', '지속력 만족도(%)', '총 용량', '타겟_공용', '타겟_남성'])
price2 = model.predict(input_data).item()
print(f"{int(price2 / 100 * 30)}원")

# 가격예측3
# 개수 3개, 평점 4.7, 리뷰수 2500개, 향/냄새 만족도 86%, 지속력 만족도 90%
# 총 용량 30ml, 남성향수
input_data = pd.DataFrame([[3, 4.7, 2500, 86, 90, 30, 0.0, 1.0]],
                          columns=['개수', '평점', '리뷰수', '향/냄새 만족도(%)', '지속력 만족도(%)', '총 용량', '타겟_공용', '타겟_남성'])
price3 = model.predict(input_data).item()
print(f"{int(price3 / 100 * 30)}원")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
72051원
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
65986원
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
72460원


# 결과
- 예측값의 오차가 너무 크다.
- 예측할 때 필요한 중요한 데이터가 누락된 것으로 판단된다.

# 대안
- 각각의 향수의 메인향에 대한 데이터를 추가로 수집하여 레이블인코딩 진행할 예정이다.